## В этом проекте посмотрим на общую статистику по отделу саппорт за 2020 год. Узнаем медиану времени первого ответа по часам в будни и выходные, рассмотрим гипотезу о том, что время первого ответа клиентам влияет на пользовательский рейтинг.
## Для февраля 2021 узнаем какое количество обращений приходится на утреннюю, вечернюю смены и их перекрестие, а также количество обращений в нерабочие часы и медиану времени ожидания ответа. Посмотрим на соотношение обращений от юзеров и лидов.


In [1]:
import pandas as pd
import plotly.express as px
from scipy import stats as st
import numpy as np

In [2]:
intercom2020 = pd.read_csv('inbox-data-export.csv')

In [3]:
intercom2020.head()

,Conversation ID,Conversation URL,Title,Conversation status,Conversation tags,Created at,Last updated at,Time Zone,Inbound/Outbound,Reopened,Closed,Conversation rating,Conversation rating remark,Email,Location,Name,Type,Participated (name),User ID,Assigned to (name),Closed by (name),Teammates participated,Time to first reply (seconds),Time to last close (seconds),Teammate replies,Channel,Source
0,19131500062955,https://app.intercom.com/a/apps/h202fg1c/inbox...,NaN,Closed,NaN,2020-11-01 07:19:36,2020-11-01 15:13:55,London,Inbound,0,1,NaN,NaN,kconnect@mailfence.com,United Kingdom,NaN,User,NaN,826481,Valerie,Valerie,Valerie,7445.0,20279.0,1,Desktop,Article
1,19131500063032,https://app.intercom.com/a/apps/h202fg1c/inbox...,NaN,Closed,NaN,2020-11-01 11:02:28,2020-11-01 12:11:21,London,Outbound,0,1,NaN,NaN,marco.luis@protonmail.com,"Windhoek, Namibia",Marco Artur Soares Marques Luis,User,Marco Artur Soares Marques Luis,561912,Renata,Renata,Renata,68.0,4133.0,1,Desktop,NaN
2,19131500062837,https://app.intercom.com/a/apps/h202fg1c/inbox...,NaN,Closed,NaN,2020-10-31 18:32:05,2020-11-01 10:30:27,London,Outbound,0,1,NaN,NaN,blacktear37@yahoo.com,"Athens, Greece",Gabriela Alina Chira,User,Gabriela Alina Chira,532836,First level support,Alina,NaN,NaN,701.0,0,Desktop,NaN
3,19131500062826,https://app.intercom.com/a/apps/h202fg1c/inbox...,NaN,Closed,NaN,2020-10-31 17:58:39,2020-11-01 09:09:53,London,Inbound,0,1,NaN,NaN,NaN,"Istanbul, Turkey",NaN,Lead,NaN,f7ce5635-00c2-4f7b-8d31-11d0100b0c03,Alina,Alina,Alina,593.0,593.0,1,Desktop,NaN
4,19131500062661,https://app.intercom.com/a/apps/h202fg1c/inbox...,NaN,Closed,**EU_card**,2020-10-31 10:47:26,2020-10-31 11:07:46,London,Inbound,0,1,NaN,NaN,NaN,"Ludhiana, India",NaN,Lead,NaN,aed96b41-cbf7-42a7-9ee9-95929d225a0f,Daniel,Daniel,Daniel,688.0,1220.0,1,Desktop,NaN


In [4]:
intercom2020['Minutes to first reply'] = intercom2020['Time to first reply (seconds)'] / 60

In [5]:
intercom2020['Time Zone'].unique()

array(['London'], dtype=object)

# Посмотрим на количество обращений по каналам связи.

In [6]:
channels = intercom2020.pivot_table(index='Channel', values='Conversation ID', aggfunc='count')
channels = channels.reset_index()
channels

,Channel,Conversation ID
0,Android App,3424
1,Desktop,27103
2,Email,3259
3,Facebook,113
4,iOS App,4064


In [7]:
px.pie(channels, names='Channel',values='Conversation ID')

In [8]:
intercom2020['Created at'] = pd.to_datetime(intercom2020['Created at'])

In [9]:
intercom2020['Last updated at'] = pd.to_datetime(intercom2020['Created at'])

In [10]:
by_source = intercom2020.pivot_table(index='Source', values='Conversation ID', aggfunc='count')
by_source

,Conversation ID
Source,
Article,247
Manual message,425
User auto message,8528


In [12]:
intercom2020.describe()

,Conversation ID,Reopened,Closed,Conversation rating,Time to first reply (seconds),Time to last close (seconds),Teammate replies,Minutes to first reply
count,3.796300e+04,37963.000000,37963.000000,4999.000000,2.880200e+04,3.707500e+04,37963.000000,28802.000000
mean,1.029124e+13,0.035772,1.391012,4.360472,4.863968e+03,6.939596e+04,2.953507,81.066132
std,9.526151e+12,0.205774,1.016095,0.994626,2.217721e+05,4.367272e+05,4.656707,3696.201216
min,2.522102e+10,0.000000,0.000000,1.000000,2.000000e+00,4.000000e+00,0.000000,0.033333
25%,2.623905e+10,0.000000,1.000000,4.000000,2.290000e+02,1.114000e+03,1.000000,3.816667
50%,1.913150e+13,0.000000,1.000000,5.000000,5.810000e+02,4.609000e+03,1.000000,9.683333
75%,1.913150e+13,0.000000,1.000000,5.000000,2.354750e+03,1.776300e+04,3.000000,39.245833
max,1.913150e+13,5.000000,24.000000,5.000000,3.685420e+07,3.685420e+07,114.000000,614236.733333


In [13]:
px.box(intercom2020, y='Minutes to first reply')

In [48]:
#Удалим выбросы
q995 = intercom2020['Minutes to first reply'].quantile(0.995)

In [15]:
intercom2020 = intercom2020[intercom2020['Minutes to first reply'] <= q995]
intercom2020 = intercom2020.reset_index()

# Изучим медиану времени первого ответа клиентам на буднях и на выходных

In [16]:
px.histogram(intercom2020, x='Minutes to first reply', range_x=(0,100))

In [19]:
intercom2020['Created_at_hour'] = intercom2020['Created at'].dt.hour

In [20]:
intercom2020['Created_at_day'] = intercom2020['Created at'].dt.dayofweek

In [21]:
intercom2020['type_of_day'] =  ['Weekend' if i > 4 else 'Business Day' for i in intercom2020['Created_at_day']]
#intercom2020

In [22]:
pivot = intercom2020.pivot_table(index=['type_of_day', 'Created_at_hour'], aggfunc={'Minutes to first reply':['mean','median']})
pivot = pivot.reset_index()

In [23]:
pivot.columns = pivot.columns.droplevel()

In [24]:
pivot.columns = ['type_of_day', 'hour', 'mean', 'median']
#pivot

In [25]:
temp = pivot[pivot['type_of_day'] == 'Business Day']
px.bar(temp, x = 'hour', y ='median', title = 'Business Day Median First Response Time') # 8 AM-7 PM UTC working hours

In [26]:
temp2 = pivot[pivot['type_of_day'] == 'Weekend']
px.bar(temp, x = 'hour', y ='mean', title = 'Weekend Median First Response Time') # 9 AM-5 PM working hours

# Проверим гипотезу о том, что время ответа больше 10 минут влияет на пользовательский рейтинг

In [27]:
group_fast = intercom2020[intercom2020['Minutes to first reply'] < 10].dropna(subset=['Conversation rating']).reset_index()
group_slow = intercom2020[intercom2020['Minutes to first reply'] > 10].dropna(subset=['Conversation rating']).reset_index()

За нулевую гипотезу возьмем утверждение, что время ответа на пользовательский рейтинг не влияет. За альтернативную, что влияет.

In [28]:
results = st.mannwhitneyu(group_fast['Conversation rating'], group_slow['Conversation rating'])

if results.pvalue < 0.05:
  print('Нулевая гипотеза может быть отвергнута')
else:
  print('Нулевая гипотеза не может быть отвергнута')

round(group_fast['Conversation rating'].mean() -  group_slow['Conversation rating'].mean(), 3)

Нулевая гипотеза может быть отвергнута


0.125

По результатам теста видим, что оценки у чатов со временем ответа больше и меньше 10 минут различны. 
Также, видим, что средняя оценка диалогов со временем первого ответа меньше 10 минут выше на 0.125.
Отсюда можем сделать вывод, что время первого ответа клиенту меньше 10 минут влияет на пользовательский рейтинг. 

В следующем тесте за нулевую гипотезу возьмем то, что время первого ответа клиенту не различается у групп с высоким и низким пользовательским рейтингом. За альтернативную возьмем, что время ответа у двух групп различается.

In [29]:
group_high_rate = intercom2020[intercom2020['Conversation rating'] > 3].dropna(subset=['Minutes to first reply']).reset_index()
group_low_score = intercom2020[intercom2020['Conversation rating'] < 3].dropna(subset=['Minutes to first reply']).reset_index()

In [30]:
results = st.mannwhitneyu(group_high_rate['Minutes to first reply'], group_low_score['Minutes to first reply'])

if results.pvalue < 0.05:
  print('Нулевая гипотеза может быть отвергнута')
else:
  print('Нулевая гипотеза не может быть отвергнута')

round(group_low_score['Minutes to first reply'].mean() -  group_high_rate['Minutes to first reply'].mean(), 1)

Нулевая гипотеза может быть отвергнута


9.2

Видим, что среднее время ответа в двух группах различно. Также, среднее время ответа в группе диалогов с низким рейтингом выше на 9.2 минуты.

# Узнаем как менялось количество обращений и среднее время ответа в течение дня в феврале

In [31]:
#таблица с количеством человек на сменах по дням
february_shifts = pd.read_csv('/content/timetable_feb.csv')
february_shifts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   День    28 non-null     object
 1   утро    28 non-null     int64 
 2   вечер   28 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 800.0+ bytes


In [32]:
february_shifts['День'] = pd.to_datetime(february_shifts['День'], format = '%m/%d/%Y')

In [33]:
#выгрузка из интеркома за февраль
intercom_feb = pd.read_csv('/content/Intercom_feb.csv')
intercom_feb

,Conversation ID,Conversation URL,Conversation status,Conversation tags,Created at,Last updated at,Reopened,Closed,Conversation rating,Email,Location,Name,Type,Participated (name),User ID,Assigned to (name),Closed by (name),Teammates participated,Time to first reply (seconds),Time to last close (seconds),Teammate replies,Channel,Source
0,19131500138140,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,NaN,2021-02-24 17:20:51,2021-02-24 18:32:06,0,1,NaN,uknomi4u@gmail.com,"Christchurch, United Kingdom",Nomeda Parker,User,Nomeda Parker,857098,Renata,Renata,Renata,123.0,1024.0,7,Desktop,User auto message
1,19131500136713,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,"B2B, *crypto_transfers*",2021-02-23 08:48:37,2021-02-24 17:48:46,0,1,NaN,NaN,"Tallinn, Estonia",NaN,Lead,NaN,72380ae5-856f-444a-a095-466d30a0e1f2,Albina,Albina,Albina,71656.0,71998.0,3,Desktop,NaN
2,19131500137984,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,"**Transaction_monitoring**, *bank_transfers*",2021-02-24 14:33:02,2021-02-24 16:08:01,0,1,NaN,jaychanel83@gmail.com,"Kaiserslautern, Germany",Cecilia Agyapong,User,Cecilia Agyapong,842469,Valerie,Valerie,Valerie,3039.0,3042.0,1,Email,NaN
3,19131500137306,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,"**Transaction_monitoring**, *bank_transfers*",2021-02-23 18:10:40,2021-02-24 17:09:02,0,1,NaN,lkodomskoi@gmail.com,"Offenburg, Germany",Leonid Kodomskoi,User,Leonid Kodomskoi,724459,Valerie,Valerie,Valerie,18025.0,29643.0,2,Email,NaN
4,19131500131163,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,**Verification**,2021-02-17 03:38:31,2021-02-24 15:34:17,0,1,NaN,wajdi.matar@hotmail.com,"Dortmund, Germany",Wajdi Matar,User,Wajdi Matar,162774,Renata,Renata,Renata,NaN,282250.0,5,Android App,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7071,19131500138478,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,NaN,2021-02-25 09:00:56,2021-02-25 09:01:57,0,1,NaN,invitations@african-kingdoms-federation.com,NaN,Mumbere Elia,Lead,Mumbere Elia,8c2c93ce-f5e7-48e2-bb8f-82dfe2315adc,First level support,NaN,NaN,NaN,NaN,0,Email,NaN
7072,19131500138479,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,NaN,2021-02-25 09:01:07,2021-02-25 09:01:57,0,1,NaN,invitations@african-kingdoms-federation.com,NaN,Mumbere Elia,Lead,Mumbere Elia,8c2c93ce-f5e7-48e2-bb8f-82dfe2315adc,Tatiana,NaN,NaN,NaN,NaN,0,Email,NaN
7073,19131500138480,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,NaN,2021-02-25 09:01:09,2021-02-25 09:01:58,0,1,NaN,invitations@african-kingdoms-federation.com,NaN,Mumbere Elia,Lead,Mumbere Elia,8c2c93ce-f5e7-48e2-bb8f-82dfe2315adc,Email (exZendesk),NaN,NaN,NaN,NaN,0,Email,NaN
7074,19131500138481,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,NaN,2021-02-25 09:01:35,2021-02-25 09:01:58,0,1,NaN,invitations@african-kingdoms-federation.com,NaN,Mumbere Elia,Lead,Mumbere Elia,8c2c93ce-f5e7-48e2-bb8f-82dfe2315adc,First level support,NaN,NaN,NaN,NaN,0,Email,NaN


In [34]:
intercom_feb['Created at'] = pd.to_datetime(intercom_feb['Created at'])

In [35]:
intercom_feb['month'] = intercom_feb['Created at'].dt.month

In [36]:
intercom_feb['Created_at_hour'] = intercom_feb['Created at'].dt.hour

In [37]:
intercom_feb['Minutes to first reply'] = intercom_feb['Time to first reply (seconds)'] / 60

In [38]:
intercom_feb['Date_only'] = intercom_feb['Created at'].dt.date
intercom_feb

,Conversation ID,Conversation URL,Conversation status,Conversation tags,Created at,Last updated at,Reopened,Closed,Conversation rating,Email,Location,Name,Type,Participated (name),User ID,Assigned to (name),Closed by (name),Teammates participated,Time to first reply (seconds),Time to last close (seconds),Teammate replies,Channel,Source,month,Created_at_hour,Minutes to first reply,Date_only
0,19131500138140,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,NaN,2021-02-24 17:20:51,2021-02-24 18:32:06,0,1,NaN,uknomi4u@gmail.com,"Christchurch, United Kingdom",Nomeda Parker,User,Nomeda Parker,857098,Renata,Renata,Renata,123.0,1024.0,7,Desktop,User auto message,2,17,2.050000,2021-02-24
1,19131500136713,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,"B2B, *crypto_transfers*",2021-02-23 08:48:37,2021-02-24 17:48:46,0,1,NaN,NaN,"Tallinn, Estonia",NaN,Lead,NaN,72380ae5-856f-444a-a095-466d30a0e1f2,Albina,Albina,Albina,71656.0,71998.0,3,Desktop,NaN,2,8,1194.266667,2021-02-23
2,19131500137984,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,"**Transaction_monitoring**, *bank_transfers*",2021-02-24 14:33:02,2021-02-24 16:08:01,0,1,NaN,jaychanel83@gmail.com,"Kaiserslautern, Germany",Cecilia Agyapong,User,Cecilia Agyapong,842469,Valerie,Valerie,Valerie,3039.0,3042.0,1,Email,NaN,2,14,50.650000,2021-02-24
3,19131500137306,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,"**Transaction_monitoring**, *bank_transfers*",2021-02-23 18:10:40,2021-02-24 17:09:02,0,1,NaN,lkodomskoi@gmail.com,"Offenburg, Germany",Leonid Kodomskoi,User,Leonid Kodomskoi,724459,Valerie,Valerie,Valerie,18025.0,29643.0,2,Email,NaN,2,18,300.416667,2021-02-23
4,19131500131163,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,**Verification**,2021-02-17 03:38:31,2021-02-24 15:34:17,0,1,NaN,wajdi.matar@hotmail.com,"Dortmund, Germany",Wajdi Matar,User,Wajdi Matar,162774,Renata,Renata,Renata,NaN,282250.0,5,Android App,NaN,2,3,NaN,2021-02-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7071,19131500138478,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,NaN,2021-02-25 09:00:56,2021-02-25 09:01:57,0,1,NaN,invitations@african-kingdoms-federation.com,NaN,Mumbere Elia,Lead,Mumbere Elia,8c2c93ce-f5e7-48e2-bb8f-82dfe2315adc,First level support,NaN,NaN,NaN,NaN,0,Email,NaN,2,9,NaN,2021-02-25
7072,19131500138479,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,NaN,2021-02-25 09:01:07,2021-02-25 09:01:57,0,1,NaN,invitations@african-kingdoms-federation.com,NaN,Mumbere Elia,Lead,Mumbere Elia,8c2c93ce-f5e7-48e2-bb8f-82dfe2315adc,Tatiana,NaN,NaN,NaN,NaN,0,Email,NaN,2,9,NaN,2021-02-25
7073,19131500138480,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,NaN,2021-02-25 09:01:09,2021-02-25 09:01:58,0,1,NaN,invitations@african-kingdoms-federation.com,NaN,Mumbere Elia,Lead,Mumbere Elia,8c2c93ce-f5e7-48e2-bb8f-82dfe2315adc,Email (exZendesk),NaN,NaN,NaN,NaN,0,Email,NaN,2,9,NaN,2021-02-25
7074,19131500138481,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,NaN,2021-02-25 09:01:35,2021-02-25 09:01:58,0,1,NaN,invitations@african-kingdoms-federation.com,NaN,Mumbere Elia,Lead,Mumbere Elia,8c2c93ce-f5e7-48e2-bb8f-82dfe2315adc,First level support,NaN,NaN,NaN,NaN,0,Email,NaN,2,9,NaN,2021-02-25


In [39]:
intercom_feb = intercom_feb.set_index('Date_only')
february_shifts = february_shifts.set_index('День')


In [40]:
#объединим таблицу из интеркома и таблицу с кол-вом человек на смене
feb_merged = pd.merge(intercom_feb, february_shifts, how='right', right_index=True, left_index=True)

In [41]:
#зададим временные промежутки (утро, пересечение смен, вечер и нерабочие часы)
feb_merged['session'] = ['morning' if i in [8,9,10] else 'both' if i in [11,12,13,14,15] \
                         else 'evening' if i in [16,17,18] else 'non working' for i in feb_merged['Created_at_hour']]
feb_merged

,Conversation ID,Conversation URL,Conversation status,Conversation tags,Created at,Last updated at,Reopened,Closed,Conversation rating,Email,Location,Name,Type,Participated (name),User ID,Assigned to (name),Closed by (name),Teammates participated,Time to first reply (seconds),Time to last close (seconds),Teammate replies,Channel,Source,month,Created_at_hour,Minutes to first reply,утро,вечер,session
2021-02-01,19131500118186,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,NaN,2021-02-01 16:24:28,2021-02-01 17:07:06,0,1,NaN,exito0007@gmail.com,Puerto Rico,NaN,User,NaN,677985,Luda,Luda,Luda,178.0,2557.0,1,Desktop,User auto message,2,16,2.966667,3,3,evening
2021-02-01,19131500118213,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,"**Transaction_monitoring**, *bank_transfers*",2021-02-01 16:37:27,2021-02-04 11:30:46,0,1,NaN,stacey@adswindowfilms.co.uk,"Plymouth, United Kingdom",NaN,User,NaN,852048,Luda,Luda,Luda,273.0,92576.0,6,Email,NaN,2,16,4.550000,3,3,evening
2021-02-01,19131500118392,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,NaN,2021-02-01 20:00:02,2021-02-08 20:06:48,0,1,NaN,leo.louison@hotmail.fr,"Sainte-Anne, Guadeloupe",Leocadie Marie Louison,User,Leocadie Marie Louison,845354,Valerie,Valerie,Valerie,14593.0,15707.0,1,Email,NaN,2,20,243.216667,3,3,non working
2021-02-01,19131500118439,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,"**Transaction_monitoring**, *bank_transfers*",2021-02-01 20:57:47,2021-02-05 14:49:46,0,2,NaN,regina.sonntag@gmx.de,"Morlenbach, Germany",NaN,User,NaN,844336,Alyona,Alyona,Alyona,26044.0,143386.0,5,Email,NaN,2,20,434.066667,3,3,non working
2021-02-01,19131500118056,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,NaN,2021-02-01 15:06:02,2021-02-01 15:14:46,0,1,NaN,jonathan_mw87@hotmail.com,"London, United Kingdom",Jonathan Glen Marsh Wheatley,User,Jonathan Glen Marsh Wheatley,852105,Maria,Maria,NaN,NaN,524.0,0,Desktop,User auto message,2,15,NaN,3,3,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-28,19131500140921,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,"**Transaction_monitoring**, *bank_transfers*",2021-02-28 17:29:46,2021-03-01 15:09:19,0,2,NaN,emmanuellaagyeman@hotmail.com,"Leicester, United Kingdom",Emmanuella Angel Abina Agyeman,User,Emmanuella Angel Abina Agyeman,861099,Alyona,Alyona,"Tatiana,Alyona",8326.0,25759.0,5,iOS App,NaN,2,17,138.766667,5,0,evening
2021-02-28,19131500140946,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,*bank_transfers*,2021-02-28 18:28:56,2021-03-01 11:53:34,0,1,NaN,seb_97122@hotmail.com,"Les Abymes, Guadeloupe",Sebastien Rotin Ficadiere,User,Sebastien Rotin Ficadiere,839908,Luda,Luda,NaN,NaN,14014.0,0,Desktop,NaN,2,18,NaN,5,0,evening
2021-02-28,19131500141040,https://app.intercom.com/a/apps/h202fg1c/inbox...,Closed,NaN,2021-02-28 22:31:01,2021-03-01 12:58:31,0,1,NaN,dakitebrahim@gmail.com,"Vitoria-Gasteiz, Spain",Brahim Dakite,User,Brahim Dakite,861272,Alyona,Alyona,Alyona,17494.0,17494.0,1,Email,NaN,2,22,291.566667,5,0,non working
2021-02-28,19131500140954,https://app.intercom.com/a/apps/h202fg1c/inbox...,Snoozed,**Verification**,2021-02-28 18:44:29,2021-03-16 14:51:50,1,1,NaN,mg_mng@hotmail.com,"Schaarbeek, Belgium",Miguel Menendez,User,Miguel Menendez,108882,Luda,Luda,Luda,18532.0,575496.0,11,Email,NaN,2,18,308.866667,5,0,evening


## Найдем количество входящих обращений в каждом периоде

In [42]:
shift_pivot = feb_merged.pivot_table(index='session', values='Conversation ID', aggfunc='count').reset_index()
shift_pivot

,session,Conversation ID
0,both,2624
1,evening,1378
2,morning,1236
3,non working,1800


In [43]:
px.bar(shift_pivot, x = 'session', y ='Conversation ID', title = 'Conversations during the day')

## Найдем среднее и медиану времени первого ответа в каждом периоде

In [44]:
mfrt_by_shift = feb_merged.pivot_table(index='session', values='Minutes to first reply', aggfunc=['mean', 'median']).reset_index()
mfrt_by_shift.columns = mfrt_by_shift.columns.droplevel()
mfrt_by_shift.columns = ['Shifts', 'Mean FRT', 'Median FRT']
mfrt_by_shift

,Shifts,Mean FRT,Median FRT
0,both,154.719509,12.300000
1,evening,165.733402,17.608333
2,morning,193.860941,53.533333
3,non working,305.273957,106.200000


In [45]:
px.bar(mfrt_by_shift, x = 'Shifts', y ='Median FRT', title = 'Median FRT during the day')

# Посмотрим на количество обращений от юзеров и от лидов и медиану времени первого ответа

In [46]:
user_lead_pivot = intercom_feb.pivot_table(index='Type', aggfunc={'Conversation ID': 'count', 'Minutes to first reply': 'median',\
                                                                  'Teammate replies':'mean'}).reset_index()
user_lead_pivot

,Type,Conversation ID,Minutes to first reply,Teammate replies
0,Lead,1072,30.025000,1.408582
1,User,6004,35.383333,2.426716


In [49]:
pivot_replied = feb_merged.pivot_table(index=['Teammate replies', 'Type'], values='Conversation ID', aggfunc='count').reset_index()
pivot_replied

,Teammate replies,Type,Conversation ID
0,0,Lead,360
1,0,User,1655
2,1,Lead,431
3,1,User,1743
4,2,Lead,133
5,2,User,827
6,3,Lead,51
7,3,User,465
8,4,Lead,36
9,4,User,324


In [52]:
px.bar(pivot_replied, x = 'Teammate replies', y ='Conversation ID', title = 'Median FRT during the day', color='Type')